# Vaccines and boosters

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np

/Users/alexleedsmatthews/.local/share/virtualenvs/notebooks-HSWNtBAS/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

In [6]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

---

## Read [CDC Historical Vax Data](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc)

In [7]:
url = "https://data.cdc.gov/api/views/unsk-b7fc/rows.csv?accessType=DOWNLOAD"

In [8]:
src = pd.read_csv(url, dtype={"MMWR_week": str, "Date": str})

In [9]:
src.columns = src.columns.str.lower().str.replace(" ", "_", regex=False)

In [11]:
src["date"] = pd.to_datetime(src["date"])

In [21]:
src.columns

Index(['date', 'mmwr_week', 'location', 'distributed', 'distributed_janssen',
       'distributed_moderna', 'distributed_pfizer', 'distributed_unk_manuf',
       'dist_per_100k', 'distributed_per_100k_5plus',
       'distributed_per_100k_12plus', 'distributed_per_100k_18plus',
       'distributed_per_100k_65plus', 'administered', 'administered_5plus',
       'administered_12plus', 'administered_18plus', 'administered_65plus',
       'administered_janssen', 'administered_moderna', 'administered_pfizer',
       'administered_unk_manuf', 'admin_per_100k', 'admin_per_100k_5plus',
       'admin_per_100k_12plus', 'admin_per_100k_18plus',
       'admin_per_100k_65plus', 'recip_administered',
       'administered_dose1_recip', 'administered_dose1_pop_pct',
       'administered_dose1_recip_5plus',
       'administered_dose1_recip_5pluspop_pct',
       'administered_dose1_recip_12plus',
       'administered_dose1_recip_12pluspop_pct',
       'administered_dose1_recip_18plus',
       'administere

In [22]:
src_slim = src[src["location"] == "US"][
    [
        "date",
        "mmwr_week",
        "location",
        "series_complete_yes",
        "series_complete_pop_pct",
        "series_complete_65plus",
        "series_complete_65pluspop_pct",
        "additional_doses",
        "additional_doses_vax_pct",
        "additional_doses_50plus",
        "additional_doses_50plus_vax_pct",
        "additional_doses_65plus",
        "additional_doses_65plus_vax_pct",
        "second_booster",
        "second_booster_50plus",
        "second_booster_50plus_vax_pct",
        "second_booster_65plus",
        "second_booster_65plus_vax_pct",
    ]
]

In [25]:
first_booster = src_slim[
    (src_slim["additional_doses_65plus"] > 0)
    & (~src_slim["additional_doses_65plus"].isna())
].sort_values("date", ascending=True)
first_booster["week_no"] = range(1, len(first_booster) + 1)
first_booster = first_booster[
    [
        "week_no",
        "additional_doses",
        "additional_doses_vax_pct",
        "additional_doses_50plus",
        "additional_doses_50plus_vax_pct",
        "additional_doses_65plus",
        "additional_doses_65plus_vax_pct",
    ]
]
first_booster.head()

,week_no,additional_doses,additional_doses_vax_pct,additional_doses_50plus,additional_doses_50plus_vax_pct,additional_doses_65plus,additional_doses_65plus_vax_pct
17390,1,4031464.0,2.2,3352688.0,3.6,2497872.0,5.5
17319,2,4363791.0,2.4,3638930.0,3.9,2736068.0,6.0
17227,3,4742750.0,2.6,3955103.0,4.3,2988820.0,6.5
17174,4,5287357.0,2.9,4415114.0,4.8,3363295.0,7.4
17139,5,5711774.0,3.1,4778321.0,5.2,3660445.0,8.0


In [28]:
primary_vax = src_slim[
    (src_slim["series_complete_65plus"] > 0)
    & (~src_slim["series_complete_65plus"].isna())
].sort_values("date", ascending=True)
primary_vax["week_no"] = range(1, len(primary_vax) + 1)
primary_vax = primary_vax[
    [
        "week_no",
        "series_complete_yes",
        "series_complete_pop_pct",
        "series_complete_65plus",
        "series_complete_65pluspop_pct",
    ]
]
primary_vax.head()

,week_no,series_complete_yes,series_complete_pop_pct,series_complete_65plus,series_complete_65pluspop_pct
31016,1,28709428,8.6,14293292,26.1
30923,2,29816907,9.0,15016305,27.4
30891,3,30790201,9.3,15626638,28.5
30822,4,31493040,9.5,16035530,29.2
30755,5,32102061,9.7,16348308,29.8


In [30]:
second_boost = src_slim[
    (src_slim["second_booster_65plus"] > 0)
    & (~src_slim["second_booster_65plus"].isna())
].sort_values("date", ascending=True)
second_boost["week_no"] = range(1, len(second_boost) + 1)
second_boost = second_boost[
    [
        "week_no",
        "second_booster",
        "second_booster_50plus",
        "second_booster_50plus_vax_pct",
        "second_booster_65plus",
        "second_booster_65plus_vax_pct",
    ]
]
second_boost.head()

,week_no,second_booster,second_booster_50plus,second_booster_50plus_vax_pct,second_booster_65plus,second_booster_65plus_vax_pct
4111,1,6065193.0,5738372.0,9.4,4210645.0,12.5
4067,2,6333401.0,6001053.0,9.9,4407959.0,13.1
3978,3,6529740.0,6192817.0,10.2,4546308.0,13.5
3910,4,6637529.0,6297783.0,10.3,4616569.0,13.7
3848,5,7230494.0,6862381.0,11.2,5011231.0,14.8


In [38]:
over65_primary_progress = src_slim[["date", "series_complete_65pluspop_pct"]].rename(
    columns={"series_complete_65pluspop_pct": "value"}
)
over65_primary_progress["type"] = "primary"
over65_primary_progress.head()

,date,value,type
56,2021-02-11,0.0,primary
212,2021-03-01,0.0,primary
248,2021-02-28,0.0,primary
283,2022-07-13,91.7,primary
402,2022-07-06,91.6,primary


In [32]:
over65_1stboost_progress = src_slim[["date", "additional_doses_65plus_vax_pct"]].rename(
    columns={"additional_doses_65plus_vax_pct": "value"}
)
over65_1stboost_progress["type"] = "1st boost"
over65_1stboost_progress.head()

,date,value,type
56,2021-02-11,0.0,1st boost
212,2021-03-01,0.0,1st boost
248,2021-02-28,0.0,1st boost
283,2022-07-13,70.4,1st boost
402,2022-07-06,70.3,1st boost


In [33]:
over65_2ndboost_progress = src_slim[["date", "second_booster_65plus_vax_pct"]].rename(
    columns={"second_booster_65plus_vax_pct": "value"}
)
over65_2ndboost_progress["type"] = "2nd boost"
over65_2ndboost_progress.head()

In [ ]:
over65_2ndboost_progress.sort_values("date", ascending=True)

In [40]:
over65_1stboost_progress = over65_1stboost_progress[
    (over65_1stboost_progress["value"] > 0)
    & (~over65_1stboost_progress["value"].isna())
]
over65_2ndboost_progress = over65_2ndboost_progress[
    (over65_2ndboost_progress["value"] > 0)
    & (~over65_2ndboost_progress["value"].isna())
]
over65_primary_progress = over65_primary_progress[
    (over65_primary_progress["value"] > 0) & (~over65_primary_progress["value"].isna())
]

progress_65plus = pd.concat(
    [over65_primary_progress, over65_1stboost_progress, over65_2ndboost_progress]
)

In [45]:
recent_progress = progress_65plus[progress_65plus["date"] >= "2022-01-01"]

In [47]:
alt.Chart(progress_65plus).mark_line().encode(
    x=alt.X("date:O"),
    y="value",
    color=alt.Color("type:N"),
)

alt.Chart(...)

In [52]:
recent_vax = progress_65plus[progress_65plus["date"] == "2022-07-13"]

In [55]:
alt.Chart(recent_vax).mark_bar().encode(
    x=alt.X("type:N"),
    y="value",
    color=alt.Color("type:N"),
).properties(width=300, height=150)

alt.Chart(...)

In [59]:
primary_vax_65plus = primary_vax[["week_no", "series_complete_65plus"]].rename(
    columns={"series_complete_65plus": "val"}
)
primary_vax_65plus["type"] = "primary"
primary_vax_65plus["newdoses"] = primary_vax_65plus["val"] - primary_vax_65plus[
    "val"
].shift(1)

In [71]:
booster_65plus = first_booster[["week_no", "additional_doses_65plus"]].rename(
    columns={"additional_doses_65plus": "val"}
)
booster_65plus["type"] = "1st boost"
booster_65plus["newdoses"] = booster_65plus["val"] - booster_65plus["val"].shift(1)
booster_65plus.head()

,week_no,val,type,newdoses
17390,1,2497872.0,1st boost,NaN
17319,2,2736068.0,1st boost,238196.0
17227,3,2988820.0,1st boost,252752.0
17174,4,3363295.0,1st boost,374475.0
17139,5,3660445.0,1st boost,297150.0


In [72]:
secondboost_65plus = second_boost[["week_no", "second_booster_65plus"]].rename(
    columns={"second_booster_65plus": "val"}
)
secondboost_65plus["type"] = "second boost"
secondboost_65plus["newdoses"] = secondboost_65plus["val"] - secondboost_65plus[
    "val"
].shift(1)
secondboost_65plus.head()

,week_no,val,type,newdoses
4111,1,4210645.0,second boost,NaN
4067,2,4407959.0,second boost,197314.0
3978,3,4546308.0,second boost,138349.0
3910,4,4616569.0,second boost,70261.0
3848,5,5011231.0,second boost,394662.0


In [73]:
newdoses_65plus_byweek = pd.concat(
    [primary_vax_65plus, booster_65plus, secondboost_65plus]
)

In [79]:
newdoses_65plus_byweek = newdoses_65plus_byweek[
    (~newdoses_65plus_byweek["newdoses"].isna())
    & (newdoses_65plus_byweek["week_no"] <= 60)
]

In [82]:
alt.Chart(newdoses_65plus_byweek).mark_line(point=True).encode(
    x=alt.X("week_no:O"),
    y="newdoses",
    color=alt.Color("type:N"),
)

alt.Chart(...)

In [78]:
secondboost_65plus

,week_no,val,type,newdoses
4111,1,4210645.0,second boost,NaN
4067,2,4407959.0,second boost,197314.0
3978,3,4546308.0,second boost,138349.0
3910,4,4616569.0,second boost,70261.0
3848,5,5011231.0,second boost,394662.0
3792,6,5325849.0,second boost,314618.0
3727,7,5517991.0,second boost,192142.0
3662,8,5725206.0,second boost,207215.0
3612,9,5918044.0,second boost,192838.0
3545,10,6048270.0,second boost,130226.0


In [81]:
dw.add_data(chart_id="TIVYB", data=recent_vax)

<Response [201]>

In [83]:
recent_vax

,date,value,type
283,2022-07-13,91.7,primary
283,2022-07-13,70.4,1st boost
283,2022-07-13,35.2,2nd boost


In [86]:
src_slim[src_slim["date"] == "2022-07-13"]

,date,mmwr_week,location,series_complete_yes,series_complete_pop_pct,series_complete_65plus,series_complete_65pluspop_pct,additional_doses,additional_doses_vax_pct,additional_doses_50plus,additional_doses_50plus_vax_pct,additional_doses_65plus,additional_doses_65plus_vax_pct,second_booster,second_booster_50plus,second_booster_50plus_vax_pct,second_booster_65plus,second_booster_65plus_vax_pct
283,2022-07-13,28,US,222682315,67.1,50240887,91.7,107004061.0,48.1,64108466.0,62.6,35358851.0,70.4,19259793.0,18246563.0,28.5,12451941.0,35.2


In [85]:
src_slim["date"].max()

Timestamp('2022-07-13 00:00:00')